In [100]:
import os
import requests as req
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
from IPython.display import Markdown, display, update_display
import json

<h1>AIRLINE ASSITANT </h1>

In [75]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."


In [76]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


In [77]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [23]:
# price_function = {
#     "name": "get_ticket_price",
#     "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "destination_city": {
#                 "type": "string",
#                 "description": "The city that the customer wants to travel to",
#             },
#         },
#         "required": ["destination_city"],
#         "additionalProperties": False
#     }
# }

In [79]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever a customer asks for a ticket price.",
    "input_schema": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [80]:
tools = [price_function]

<h1>LET ANTHROPIC TO USE OUR TOOL </h1>

In [85]:
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY in .env")

from anthropic import Anthropic
client = Anthropic(api_key=api_key)
MODEL = "claude-sonnet-4-20250514"

In [99]:
def chat(message, history):
    anthro_messages = []
    for m in history:
         
         if m["role"] in ("user", "assistant"):
             
             anthro_messages.append({
                "role": m["role"],
                "content": [{"type": "text", "text": m["content"]}]
            })
                
   
     

   
    anthro_messages.append({"role": "user", "content": [{"type": "text", "text": message}]})


   
    response = client.messages.create(
        model=MODEL,
        max_tokens=500,
        system=system_message,
        tools=tools,
        messages=anthro_messages
    )

    tool_results = []
    for block in response.content:
        if block.type == "tool_use":
            tool_name = block.name
            tool_args = block.input
            tool_id = block.id
            # Execute the local tool
            result = get_ticket_price(tool_args["destination_city"])
            # Prepare tool_result to send back
            tool_results.append({
                "tool_use_id": tool_id,
                "content": json.dumps({"destination_city": tool_args["destination_city"], "price": result})
            })

   
    if not tool_results:
        
        return response.content[0].text

    # Step 2: send tool results back so Claude can finish
    followup = client.messages.create(
        model=MODEL,
        max_tokens=500,
        system=system_message,
        tools=tools,
        messages=[
            *anthro_messages,
            {"role": "assistant", "content": response.content},
            {"role": "user", "content": [
                {"type": "tool_result", "tool_use_id": tr["tool_use_id"], "content": tr["content"]}
                for tr in tool_results
            ]}
        ]
    )

    final_text = "".join(b.text for b in followup.content if b.type == "text")
    return final_text






In [104]:
gr.ChatInterface(fn=chat, type="messages").launch(share = True)

* Running on local URL:  http://127.0.0.1:7883
* Running on public URL: https://6ea7abcad164270221.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool get_ticket_price called for Berlin


<h1>Let OLLAMA TO USE TOOL </h1>

In [101]:
def chat_ollama(message, history):
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": message}]
    )

    resp = req.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3",
            "messages": messages,
            "stream": False
        }
    )

    data = resp.json()
    reply = data["message"]["content"] 
    # Check for tool call
    if "tool_calls" in data["message"] and len(data["message"]["tool_calls"]) > 0:
        tool_response, city = handle_tool_call_ollama(data["message"])
        messages.append(data["message"])  
        messages.append(tool_response)   
        
        resp2 = req.post(
            "http://localhost:11434/api/chat",
            json={
                "model": "llama3",
                "messages": messages,
                "stream": False
            }
        )
        data2 = resp2.json()
        reply = data2["message"]["content"]
    
    return reply

In [102]:
def handle_tool_call(message):
    tool_call = message["tool_calls"][0]
    arguments = json.loads(tool_call["function"]["arguments"])
    city = arguments.get("destination_city")
    price = get_ticket_price(city)  # your function to fetch price
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call["id"]
    }
    return response, city

In [103]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin
